<a href="https://colab.research.google.com/github/Siddhant-Thendral-Arasu/Siddhant-Thendral-Arasu/blob/chest_xray_classification/Chest_xray_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classifying Chest XRay images using convolutional block, identity block, TensorFlow/Keras

In [19]:
from google.colab import drive

drive.mount(r'/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [20]:
with h5py.File(r'/content/drive/My Drive/chest_xray/ChestXray14_preprocessed.h5') as h5f:
  print("keys:", list(h5f.keys()))
  x_data = np.array(h5f['images'])  # Image data
  y_data = np.array(h5f['labels'])  # Labels
# Print shapes for confirmation
print(f"x_data shape: {x_data.shape}")  # Should be (num_samples, 224, 224, 1)
print(f"y_data shape: {y_data.shape}")  # Should be (num_samples, 15)

# Split into training and validation sets (e.g., 80% train, 20% validation)
x_train, x_val, y_train, y_val = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42, stratify=y_data)

# Print final split shapes for confirmation
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")

keys: ['images', 'labels']
x_data shape: (4999, 224, 224, 1)
y_data shape: (4999, 4)
x_train shape: (3999, 224, 224, 1)
y_train shape: (3999, 4)
x_val shape: (1000, 224, 224, 1)
y_val shape: (1000, 4)


In [22]:
# Normalize images
x_train = x_train / 255.0

In [23]:
from tensorflow.keras.layers import Flatten, Conv2D, BatchNormalization, ReLU, Add, Input, Dense, Input
from tensorflow.keras.models import Model

def convolutional_block(X, filters, kernel_size, strides=(2, 2)):
    """
    Convolutional block with skip connection.
    Args:
        X: Input tensor
        filters: Tuple of integers, the number of filters for the Conv2D layers
        kernel_size: Size of the convolutional kernel
        strides: Strides for the first convolution
    Returns:
        Output tensor
    """
    F1, F2, F3 = filters

    # Main path
    X_shortcut = X

    X = Conv2D(F1, kernel_size=(1, 1), strides=strides, padding='valid')(X)
    X = BatchNormalization()(X)
    X = ReLU()(X)

    X = Conv2D(F2, kernel_size=kernel_size, strides=(1, 1), padding='same')(X)
    X = BatchNormalization()(X)
    X = ReLU()(X)

    X = Conv2D(F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization()(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, kernel_size=(1, 1), strides=strides, padding='valid')(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)

    # Add shortcut and main path
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X



def identity_block(X, filters, kernel_size):
    """
    Identity block with skip connection.
    Args:
        X: Input tensor
        filters: Tuple of integers, the number of filters for the Conv2D layers
        kernel_size: Size of the convolutional kernel
    Returns:
        Output tensor
    """
    F1, F2, F3 = filters

    # Main path
    X_shortcut = X

    X = Conv2D(F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization()(X)
    X = ReLU()(X)

    X = Conv2D(F2, kernel_size=kernel_size, strides=(1, 1), padding='same')(X)
    X = BatchNormalization()(X)
    X = ReLU()(X)

    X = Conv2D(F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization()(X)

    # Add shortcut and main path
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X


def resnet50(input_shape=(224, 224, 1), classes=4):
    """
    Implementation of ResNet50.
    Args:
        input_shape: Shape of input images
        classes: Number of output classes
    Returns:
        ResNet50 model
    """
    X_input = Input(input_shape)

    # Stage 1
    X = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(X_input)
    X = BatchNormalization()(X)
    X = ReLU()(X)

    # Stage 2
    X = convolutional_block(X, filters=(64, 64, 256), kernel_size=(3, 3), strides=(1, 1))
    X = identity_block(X, filters=(64, 64, 256), kernel_size=(3, 3))

    # Stage 3
    X = convolutional_block(X, filters=(128, 128, 512), kernel_size=(3, 3))
    X = identity_block(X, filters=(128, 128, 512), kernel_size=(3, 3))

    # Flatten and output
    X = Conv2D(classes, kernel_size=(1, 1))(X)
    X = ReLU()(X)
    X = BatchNormalization()(X)

    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X)

    return model


In [24]:
# Create model
model = resnet50(input_shape=(224, 224, 1), classes=4)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(x_train, y_train, batch_size=32, epochs=1, validation_split=0.2)

100/100 ━━━━━━━━━━━━━━━━━━━━ 4907s 49s/step - accuracy: 0.4649 - loss: 0.9144 - val_accuracy: 0.3050 - val_loss: 1.5345


In [7]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 112, 112, 64)   │          3,200 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 112, 112, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 112, 112, 64)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 112, 112, 64)   │          4,160 │ re_lu[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 112, 112, 64)   │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 112, 112, 64)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 112, 112, 64)   │         36,928 │ re_lu_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 112, 112, 64)   │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 112, 112, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 112, 112, 256)  │         16,640 │ re_lu_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 112, 112, 256)  │         16,640 │ re_lu[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 112, 112, 256)  │          1,024 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 112, 112, 256)  │          1,024 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 112, 112, 256)  │              0 │ batch_normalization_3… │
│                           │                        │                │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_3 (ReLU)            │ (None, 112, 112, 256)  │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)    

 Total params: 869,784 (3.32 MB)

 Trainable params: 863,504 (3.29 MB)

 Non-trainable params: 6,280 (24.53 KB)

Kaggle dataset with 100K+ images
https://www.kaggle.com/datasets/nih-chest-xrays/data

Due to hugh dataset size and larger training time, .h5 is created with reduced number of classes and images resulting in lower accuracy. This tiny dataset of 5K images took about 2 hours for training with .h5 mounted onto google drive and the original dataset size (100K+) may take about 2 days.